In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
columbia = pd.read_csv("MasterSheet CSVS/Columbia.csv")# student ig, candy, medical conditions
ephrata = pd.read_csv("MasterSheet CSVS/Ephrata.csv")#comments, medical conditions 
lcath = pd.read_csv("MasterSheet CSVS/Lancaster Catholic.csv")#medical conditions, comments
manCent = pd.read_csv("MasterSheet CSVS/Manheim Central.csv")#medical conditions, comments
manTown = pd.read_csv("MasterSheet CSVS/Manheim Township.csv")#comments
penn = pd.read_csv("MasterSheet CSVS/Penn Manor.csv")#20 columns
total = [columbia, ephrata, lcath, manCent, manTown, penn]

In [ ]:
def pcol(df):
    print(df.columns.size, df.columns)
# for i in total:
#     pcol(i)
allcols = columbia.columns.tolist()
allcols.append("Comments")

In [ ]:
count =1
for df in total: 
    print(count)
    for c in allcols:
        if c not in df.columns:
            df[c]=np.nan
    df=df.reindex(allcols, axis ="columns")
    count+=1
allcols

In [ ]:
print(manTown.columns, columbia.columns)

In [ ]:
schools = ["Columbia", "Ephrata", "Lancaster Catholic", "Manheim Central", "Manheim Township", "Penn Manor"]
for i in range(len(total)):
    total[i]["School"] = schools[i]
alldf = pd.concat(total, ignore_index =True)
alldf

In [ ]:
def splitname(df, name):
    df1 = df[name].str.split(" ",expand=True,)
    df1.columns = ['{}{}'.format(name, x+1) for x in df1.columns]
    df = df.join(df1)
    name2 = name+'2'
    name3 = name+'3'
    name4 = name+'4'
    if name3 in df:
        check = df[df[name3].notnull()].index.tolist()
        print(name3, '-',check)
        
        for i in check:
            x = df.iloc[i, :]
            x[name2] = x[name2] + ' ' + x[name3]
            df.iloc[i, :] = x
        del df[name3] 
    if name4 in df:
        check = df[df[name4].notnull()].index.tolist()
        print(name4, '-', check)
        
        for i in check:
            x = df.iloc[i, :]
            x[name2] = x[name2] + ' ' + x[name4]
            df.iloc[i, :] = x
        del df[name4]
    return df

def cleanshirtsize(df, var):
    df[var] =df[var].str.lower() 
    df[var]= df[var].str.replace('large', 'l').str.replace('medium','m').str.replace('small','s').str.replace('médium','m')
    df[var]= df[var].str.replace('lx', 'xl').str.replace('x-l', 'xl').str.replace('2xl', 'xxl').replace('adultl', 'l').replace('2x','xxl')
    df[var]= df[var].str.replace('women’sxl', 'xl').str.replace('3xl','xxl').str.replace('m/l','l').str.replace('mors','m')
    df[var]= df[var].str.upper()
    return df

def cleanrace(df):
    v = 'Race'
    df[v] = df[v].str.lower().str.title().str.replace('Caucasian,White', 'White', regex=True)
    df[v]= df[v].str.replace('White,Caucasian', 'White', regex=True)
    return df

def transform(dforig):
    df = dforig.copy()
    
    #make all lower email
    df['Student Email'] = df['Student Email'].str.lower()
    df['Guardian 1 Email'] = df['Guardian 1 Email'].str.lower()
    df['Guardian 2 Email'] = df['Guardian 2 Email'].str.lower()
    #remove all nonnumbers in phonenumbers
    df['Student Phone'] = df['Student Phone'].str.replace('-', '').str.replace(' ', '')
    df['Guardian 1 Phone'] = df['Guardian 1 Phone'].str.replace('-', '').str.replace(' ', '')
    df['Guardian 2 Phone'] = df['Guardian 2 Phone'].str.replace('-', '').str.replace(' ', '')
    df['Emergency Contact'] = df['Emergency Contact'].str.replace('-', '')
    # remove all questions in sizes
    df['Shirt Size'] = df['Shirt Size'].str.replace('?', '', regex=True).str.replace(' ', '', regex=True)
    df['Short Size'] = df['Short Size'].str.replace('?', '', regex=True).str.replace(' ', '', regex=True)
    df['Guardian 1 Shirt Size'] = df['Guardian 1 Shirt Size'].str.replace('?', '', regex=True).str.replace(' ', '', regex=True)
    
    df['bday'] = df['Birthdate'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
    print(df[df.bday.isnull()]['Birthdate'])
    
    medcheck = df[df["Medical Conditions"].notnull()].index.tolist()
        
        
    for i in medcheck:
        x = df.iloc[i, :]
        if pd.isna(x["Comments"]):
            x["Comments"] = 'Medical Conditions: ' + x["Medical Conditions"]
        else:
            print(x["Comments"])
            x["Comments"] = x["Comments"] + ' Medical Conditions: ' + x["Medical Conditions"]
        df.iloc[i, :] = x
    del df["Medical Conditions"] 
    
    #Split Names
    df = splitname(df, "Student Name")   
    df = splitname(df, "Guardian 1 Name")      
    df = splitname(df, "Guardian 2 Name")      
    
    df = cleanshirtsize(df, 'Guardian 1 Shirt Size')
    df = cleanrace(df)
    return df
transdf = transform(alldf)

In [ ]:
# for i in range(len(transdf)):
#     print(transdf.loc[i,'Guardian 1 Name'], transdf.loc[i,'Guardian 1 Phone'],'-',transdf.loc[i,'Guardian 2 Name'], transdf.loc[i,'Guardian 2 Phone'],'-',transdf.loc[i,'Emergency Contact'])
print(transdf[transdf['Guardian 2 Name1'].isnull()]['Guardian 2 Name1'])
print(transdf[transdf['Guardian 2 Name2'].isnull()]['Guardian 2 Name2'])
print(transdf.loc[101,:])

In [ ]:
orderedcols = {'Student Name1':'fname', 'Student Name2':'lname', 'Student Email':'email', 
               'Student Phone':'phonenum','bday':'dob','School':'schoolid','Gender':'gender','Race':'race',
               'Address':'address','Shirt Size':'shirt','Short Size':'short','Student IG':'student_ig',
               'Favorite Candy':'favcandy','Guardian 1 Name1':'guard1fname','Guardian 1 Name2':'guard1lname',
               'Guardian 1 Phone':'guard1phonenum','Guardian 1 Email':'guard1email','Guardian 1 Occupation':'guard1occ',
               'Guardian 1 Shirt Size':'guard1shirt','Guardian 2 Name1':'guard2fname','Guardian 2 Name2':'guard2lname',
               'Guardian 2 Phone':'guard2phonenum','Guardian 2 Email':'guard2email','Guardian 2 Occupation':'guard2occ',
               'Guardian 2 Shirt Size':'guard2shirt','Emergency Contact':'emergcontact','Comments':'comments'}

In [ ]:
transdf.columns

In [ ]:
def cleantransdf(df):
    df.rename(orderedcols, axis=1, inplace=True)
cleantransdf(transdf)

In [ ]:
transdf.head()

In [ ]:
def intphone(df, var):
    df[var] = '+1'+df[var]
intphone(transdf, 'phonenum')
intphone(transdf, 'guard1phonenum')
intphone(transdf, 'guard2phonenum')

In [ ]:
filepath = 'attollo/basic/management/commands/transdf2022.csv'
transdf.to_csv(filepath)

In [ ]:
transdf = pd.read_csv(filepath)

In [ ]:
transdf